In [1]:
import requests
import xmltodict
import urllib, urllib.request
import mysql.connector
import time
from tqdm import tqdm
from IPython.display import clear_output
import uuid

In [6]:
# Get general data
link = "https://api.semanticscholar.org/graph/v1/paper/search?query=Integrating E Commerce and Data Mining: Architecture and Challenges"
headers = {"x-api-key": "M7HSjQNeTfai6l7JUiDZB8XYc85BHnHt3R0NXSEd"}
response = requests.get(link, headers=headers)
response.json()

{'total': 21,
 'offset': 0,
 'next': 10,
 'data': [{'paperId': 'fc95099a18816f9b79e202154f6073aba780b711',
   'title': 'Integrating e-commerce and data mining: architecture and challenges'},
  {'paperId': '7a0f291e2ea475e5c0dfd1497549e8dbba81c577',
   'title': 'Integrating E-Commerce & Data Mining Architecture Challenges'},
  {'paperId': '0f494865fdc6cf6d1b4a8bdfd8ee40e07642f6a8',
   'title': 'E-Commerce and Data Mining: Architecture and Challenges'},
  {'paperId': '0789c1934194a2abf9cfe5d6d603b1190b648dee',
   'title': 'Knowledge Management in E-commerce: A Data Mining Perspective'},
  {'paperId': 'd3f1edc86326306bd4b46565acc6dcd8e596dd25',
   'title': 'Void Mark E-Commerce Application'},
  {'paperId': '1cc8410934c27733a2119c378e98b46551a69d9d',
   'title': 'Big Data Mining: A Comprehensive Analysis'},
  {'paperId': '7b51e757b388f0261066ba676f41a878040bfd55',
   'title': 'A Study of Various Varieties of Distributed Data Mining Architectures'},
  {'paperId': '0428dc9c0fcd61dca17c3b4b52

In [138]:
# Get Paper data
paper_link = "https://api.semanticscholar.org/graph/v1/paper/781ef2bbb48f9ce81f8cc949c3a8a55033e6ced8?fields=url,year,referenceCount,citationCount,publicationTypes,journal,authors"
response2 = requests.get(paper_link, headers=headers)
print(response2.json())

{'paperId': '781ef2bbb48f9ce81f8cc949c3a8a55033e6ced8', 'url': 'https://www.semanticscholar.org/paper/781ef2bbb48f9ce81f8cc949c3a8a55033e6ced8', 'year': 2021, 'referenceCount': 0, 'citationCount': 112, 'publicationTypes': None, 'journal': {'name': 'Sociedade & Natureza'}, 'authors': [{'authorId': '144791575', 'name': 'R. Rosa'}, {'authorId': '40403988', 'name': 'Samuel do Carmo Lima'}, {'authorId': '134857638', 'name': 'W. Assunção'}]}


In [15]:
# Get Author data
author_link = "https://api.semanticscholar.org/graph/v1/author/10007273?fields=url"
response3 = requests.get(author_link, headers=headers)
response3.json()

{'authorId': '10007273',
 'url': 'https://www.semanticscholar.org/author/10007273'}

# Fetch data with "Artificial Intelligence" as search query from 2018

In [16]:
YEAR_START = 2020
YEAR_END = 2020
STEP_SIZE = 100
CATEGORY = "cs.CV"

In [17]:
# Establish database connection
cnx = mysql.connector.connect(user='david', password='daviddung1993',
                              host='127.0.0.1',
                              database='computervision')
cursor = cnx.cursor()

In [18]:
def store_authors(author_infomration, paperID):
    for author in author_infomration:
        affiliations = ""
        params = ()
        if len(author.keys()) == 2:
            params = (author["authorId"], author["name"], affiliations, None, None)
        else:
            if author["affiliations"]:
                affiliations = author["affiliations"][0]
            params = (author["authorId"], author["name"], affiliations, author["paperCount"], author["hIndex"])
        insert_author = ("INSERT INTO Authors VALUES (%s,%s,%s,%s,%s)")
        insert_authoredBy = ("INSERT INTO authoredBy VALUES (%s,%s)")
        try:
            cursor.execute(insert_author, params)
        except mysql.connector.IntegrityError as err:
            #print("Author exists already")
            pass
        try:
            cursor.execute(insert_authoredBy, (paperID, author["authorId"]))
        except mysql.connector.IntegrityError as err:
            #print("AuthordByEntry exists already")
            pass





def store_paper(archive_url, paper_response, isLeaf):
    isConference = False
    isJournalArticle = False
    isReview = False
    journal = ''
    if not paper_response['paperId']:
        paper_response['paperId'] = '11111' + uuid.uuid4().hex
    if paper_response["publicationTypes"]:
        if "JournalArticle" in paper_response["publicationTypes"]:
            isJournalArticle = True
        if "Conference" in paper_response["publicationTypes"]:
            isConference = True
        if "Review" in paper_response["publicationTypes"]:
            isReview = True
    if paper_response["journal"]:
        if "name" in paper_response["journal"]:
            journal = paper_response["journal"]["name"]
    params = (paper_response['paperId'], paper_response['title'], archive_url, paper_response['year'], CATEGORY, isReview, isConference, isJournalArticle,paper_response['referenceCount'], paper_response['citationCount'], journal, isLeaf, paper_response['url'])
    insert_paper = ("INSERT INTO Papers VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)")
    try:
        cursor.execute(insert_paper, params)
        return True
    except mysql.connector.IntegrityError as err:
        check_if_leaf = ("SELECT Leaf FROM Papers WHERE PaperID = %s")
        cursor.execute(check_if_leaf, [paper_response['paperId']])
        isLeafPaper = cursor.fetchone()[0]
        if isLeafPaper == isLeaf:
            return False
        update_leaf = "UPDATE Papers SET Leaf = %s WHERE PaperID = %s"
        cursor.execute(update_leaf, (isLeaf,paper_response['paperId']))
    except Exception as err:
        print(params)
        print(err)
        raise ValueError

def store_reference(paperId, referenced_by):
    try:
        insert_reference = ("INSERT INTO referencedBy VALUES (%s,%s,0)")
        cursor.execute(insert_reference, (paperId, referenced_by))
    except mysql.connector.IntegrityError as err:
        pass


In [20]:
for i in range(YEAR_END - YEAR_START + 1):
    year_temp = YEAR_START + i
    print(year_temp)
    current_index = 0
    while True:
        # Get title of all entries in a year from arxiv
        try:
            arxiv_query = f'https://export.arxiv.org/api/query?search_query=cat:cs.CV+AND+submittedDate:[{year_temp}01010630+TO+{year_temp}12311645]&sortBy=submittedDate&start={current_index}&max_results={STEP_SIZE}'
            data = urllib.request.urlopen(arxiv_query)
            data_xml = data.read().decode('utf-8')
            data_dict = xmltodict.parse(data_xml)

            total_entries = int(data_dict['feed']['opensearch:totalResults']['#text'])
            entries = data_dict['feed']['entry']
            print(f"{current_index}/{total_entries}")
            current_index += STEP_SIZE
            # Fetch information for papers from semantic scholar
            for entry in entries:
                start = time.time()
                archive_link = entry["id"]
                entry_title = entry["title"].replace("-", " ").replace("\n", "")
                semantic_scholar_query = f"https://api.semanticscholar.org/graph/v1/paper/search?query={entry_title}"
                response = requests.get(semantic_scholar_query, headers=headers).json()
                if not response["total"]:
                    continue
                else:
                    # Store papers' information in database
                    paperId = response["data"][0]["paperId"]
                    paper_url = f"https://api.semanticscholar.org/graph/v1/paper/{paperId}?fields=url,year,referenceCount,citationCount,publicationTypes,journal,authors,title,references.url,references.title,references.year,references.referenceCount,references.citationCount,references.authors,references.journal,references.publicationTypes,authors.name,authors.affiliations,authors.paperCount,authors.hIndex"
                    paper_response = requests.get(paper_url, headers=headers).json()
                    success = store_paper(archive_link, paper_response, False)
                    if not success:
                        continue
                    store_authors(paper_response['authors'], paperId)
                    # Store references' information in database
                    for reference in paper_response["references"]:
                        store_paper(None, reference, True)
                        store_reference(reference["paperId"], paperId)
                        if reference["authors"]:
                            #print(reference["authors"])
                            store_authors(reference["authors"], reference["paperId"])
            cnx.commit()
        except KeyError:
            print("sleeping")
            time.sleep(3)
        if current_index > total_entries:
            break
    clear_output(wait=True)


2020
0/15321
100/15321
200/15321
sleeping
300/15321
400/15321
500/15321
600/15321
700/15321
800/15321
900/15321
1000/15321
sleeping
1100/15321
sleeping
1200/15321
1300/15321
1400/15321
1500/15321
1600/15321
1700/15321
1800/15321
1900/15321
sleeping
2000/15321
2100/15321
2200/15321
sleeping
2300/15321
2400/15321
2500/15321
2600/15321
2700/15321
2800/15321
2900/15321
sleeping
sleeping
sleeping
3000/15321
3100/15321
3200/15321
sleeping
sleeping
sleeping
sleeping
3300/15321
sleeping
sleeping
sleeping
sleeping
sleeping
sleeping
sleeping
sleeping
sleeping
sleeping
sleeping
sleeping
sleeping
sleeping
3400/15321
3500/15321
sleeping
sleeping
3600/15321
3700/15321
sleeping
3800/15321
sleeping
sleeping
3900/15321
sleeping
sleeping
4000/15321
4100/15321
4200/15321
4300/15321
sleeping
4400/15321
4500/15321
4600/15321
4700/15321
sleeping
sleeping
4800/15321
4900/15321
sleeping
5000/15321
sleeping
sleeping
5100/15321
5200/15321
sleeping
sleeping
5300/15321
5400/15321
sleeping
5500/15321
sleeping
slee

In [ ]:
cnx.commit()

In [135]:
cursor.close()
cnx.close()

In [57]:
add_employee = ("SELECT p. FROM Papers p,referencedBy r, Papers p2 WHERE ")
try:
    cursor.execute(add_employee, ["d8ed6678758f9200bd23fcf11dd733c8f4d9d71c"])
except mysql.connector.IntegrityError as err:
    print("entry exists already")

In [58]:
cursor.fetchone()[0]

1

In [42]:
print("halo")
clear_output(wait=True)

halo


In [89]:
x = None
if x:
    print("hallo")